# TensorFlow Model Testing
This notebook tests TensorFlow installation and evaluates a simple model's accuracy on the MNIST dataset.

In [1]:
import os
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

2025-04-09 15:15:08.618610: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-09 15:15:08.794226: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-09 15:15:08.959875: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744204509.107322   13426 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744204509.149474   13426 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744204509.452000   13426 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Suppress TensorFlow GPU warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
# Set memory growth for GPU
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    try:
        for gpu in physical_devices:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

E0000 00:00:1744204532.218357   13426 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1744204532.221417   13426 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


## Test TensorFlow Installation
This function verifies that TensorFlow is installed and working correctly by performing a simple addition operation.

In [4]:
def test_tensorflow_installation():
    try:
        a = tf.constant(5)
        b = tf.constant(3)
        c = tf.add(a, b)
        print(f"TensorFlow addition result: {c.numpy()}")
        print("TensorFlow installation is working correctly.")
    except Exception as e:
        print(f"TensorFlow installation test failed: {e}")

# Run the installation test
test_tensorflow_installation()

TensorFlow addition result: 8
TensorFlow installation is working correctly.


## Define the Model Testing Function
This function evaluates a TensorFlow model on a given test dataset and checks if the accuracy meets a specified threshold.

In [5]:
def test_model(model, test_dataset, threshold=0.9):
    """
    Test a TensorFlow model on a given test dataset.

    Parameters:
        model (tf.keras.Model): The TensorFlow model to test.
        test_dataset (tf.data.Dataset): Dataset for testing the model.
        threshold (float): The accuracy threshold to determine success.

    Returns:
        float: The accuracy of the model on the test dataset.
        bool: Whether the test passed the accuracy threshold.
    """
    loss, accuracy = model.evaluate(test_dataset)
    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")
    return accuracy, accuracy >= threshold

## Load and Preprocess the MNIST Dataset
The MNIST dataset is used for testing. The data is normalized to improve model performance.

In [6]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

## Define and Train the Model
A simple feedforward neural network is created and trained on the MNIST training dataset.

In [13]:
# Define the model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(train_dataset, epochs=5)

Epoch 1/5


/home/xgen0-ubuntu/.cache/pypoetry/virtualenvs/tensorflow-models-Ij36o141-py3.11/lib/python3.11/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8795 - loss: 0.4300
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9618 - loss: 0.1316
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9751 - loss: 0.0857
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9827 - loss: 0.0618
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9875 - loss: 0.0470


## Test the Model
Evaluate the model on the test dataset and check if it meets the accuracy threshold.

In [14]:
# Test the model
accuracy, passed = test_model(model, test_dataset)
print(f"Model accuracy: {accuracy:.2f}, Passed: {passed}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9654 - loss: 0.1125
Test Loss: 0.0960, Test Accuracy: 0.9696
Model accuracy: 0.97, Passed: True
